In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
import re
import os

In [2]:
urls = ["https://gis.ngdc.noaa.gov/arcgis/rest/services/passive_acoustic_data/MapServer/0/query?where=1%3D1&outFields=*&returnGeometry=true&f=pjson",
       "https://gis.ngdc.noaa.gov/arcgis/rest/services/passive_acoustic_data/MapServer/1/query?where=1%3D1&outFields=*&returnGeometry=true&f=pjson",
       "https://gis.ngdc.noaa.gov/arcgis/rest/services/passive_acoustic_data/MapServer/2/query?where=1%3D1&outFields=*&returnGeometry=true&f=pjson",
       "https://gis.ngdc.noaa.gov/arcgis/rest/services/passive_acoustic_data/MapServer/3/query?where=1%3D1&outFields=*&returnGeometry=true&f=pjson"]

In [8]:

dict_save = []

for url in urls:
        
    # store the response of URL
    url_output = urlopen(url)

    # storing the JSON response from url in data
    data_json = json.loads(url_output.read())

    for data in data_json['features']:
        metadata = data['attributes']
        wanted_keys = ['DATA_COLLECTION_NAME','SOURCE_ORGANIZATION','PROJECT_NAME','MAX_SAMPLE_RATE','MAX_DURATION','MAX_BOTTOM_DEPTH','MAX_SENSOR_DEPTH','NUMBER_CHANNELS','geometry'] # The keys you want
        tmp = dict((k, metadata[k]) for k in wanted_keys if k in metadata)
        geometry = data['geometry']
        data_quality = metadata['DATA_QUALITY']
        sampling_details = metadata['SAMPLING_DETAILS']
        
        try:
            dates = sampling_details.split(',')[0].split(':')[2].replace(' ','')
            startDate = re.search('"(.*)to', dates).group(1)
            endDate = re.search('to(.*)"', dates).group(1)
        except:
            dates = sampling_details.split('"Date Range"')[1].split(',')[0].replace(' ','')
            startDate = re.search('"(.*)to', dates).group(1)[0:10]
            endDate = re.search('to(.*)"', dates).group(1)[0:10]
            
        tmp['LON'] = geometry['points'][0][0]
        tmp['LAT'] = geometry['points'][0][1]
        tmp['START_DATE'] = startDate
        tmp['END_DATE'] = endDate

        dict_save.append(tmp)
        
path = os.getcwd()
df = pd.DataFrame(dict_save)
df.to_csv('NCEI.csv')